In [1]:
import numpy as np
import pandas as pd
import re
import os
from IPython.display import display
from tabulate import tabulate
# import functions
import openpyxl
from openpyxl import load_workbook

In [4]:
class HMI_FLAT_FILE:
    
    def __init__(self):
        pass
    
    def to_numeric(self,df,column):
        df[column]=pd.to_numeric(df[column],errors='coerce')
        return df 

    def read_file(self,file_name,col,product_name):
        df=pd.read_parquet(file_name)
        # convert all column names to lower case.
        df.columns = df.columns.str.lower()
        # filter batches
        #print(df[df['product_desc'] == product_name])
    #     print(df[df['parameter_desc'] == 'AMLO BESY 5 MG AND TELMI 40 MG BILAYERED TABLETS'])
        df = df[df[col] == product_name]
        return df
    
    def parameter(self,df,colname,cols):
        #df=pd.read_parquet(file_name)    
         # filter batches
        #print(df[df['parameter_no'] == parameter_no])
        df = df[df[colname].isin(cols)]
        return df
    
    def group_by(self,df,name,cols):
        list_of_cols = {}
        for col in cols:
            list_of_cols[col] ='last'
        print(list_of_cols)
    #     agg_func_selection = list_of_cols
        df=df.groupby(name).agg(list_of_cols).reset_index()
        return df
    
    def pivot_table(self,df,values,index,columns):
    #     pivot_values=['list_of_cols'],index=['list_of_cols'],columns=[]
        df=pd.pivot_table(df,values=values,index=index,columns=columns)
        return df
    
    def clean_up(self,df):
        df.columns = [' '.join(col) for col in df.columns.values]
        return df



In [52]:
class APQR_FLAT_FILE:
    
    def __init__(self):
        pass
    def apqr_flat_file(self,APQR_FILE , apqr_output_file):
        df = pd.read_excel(APQR_FILE)
        List_of_MIC = ["LOSS ON DRYING","BULK DENSITY","TAPPED DENSITY","% FINES"]

        df = df[df["MIC Description"].isin(List_of_MIC)]

        pivot = pd.DataFrame()
        df_pivot = pd.pivot_table(df,index = 'Batch No.', columns = ["MIC Description"], values = ['Result' ])
        print(df_pivot.columns.values)
        df_pivot.columns = [ value[1]   for value in df_pivot.columns.values]
        pivot = pd.concat([pivot, df_pivot], axis = 1)
        
        pivot = apqr_df.reset_index().rename(columns = {'Batch No.' : 'batch_id'})
        pivot.to_excel(apqr_output_file)
        return pivot
    
    def merge(self,df_hmi,df_apqr):
        final_flat_file = df_hmi.merge(df_apqr,left_on='batch_id',right_on='batch_id',validate='1:1')
        final_flat_file.to_excel(final_output_file)  
        
        return final_flat_file


In [63]:
def DataPreprocessingOE(file_name ,product_name,parameter_no,APQR_FILE_PATH,Merged_File,hmi_output_file,apqr_output_file,final_output_file):
    
    hmi = HMI_FLAT_FILE()
    df = hmi.read_file(file_name,'product_desc','AMLO BESY 5 MG AND TELMI 40 MG BILAYERED TABLETS')
    df = hmi.to_numeric(df,"parameter_no")
    df = hmi.parameter(df,"parameter_no",parameter_no)
    print(df.shape)

    # df = to_numeric("parameter_no")
    df = hmi.to_numeric(df,"station_1_reading_set")
    df = hmi.to_numeric(df,"station_2_reading_set")
    name = ['batch_id', 'parameter_desc']
    df = hmi.group_by(df,name,['station_1_reading_set','station_2_reading_set'])
    values=['station_1_reading_set','station_2_reading_set']
    index=['batch_id']
    columns=['parameter_desc']
    df=hmi.pivot_table(df,values,index,columns)
    df = hmi.clean_up(df)
    
    df = hmi_df.reset_index()
    
    apqr = APQR_FLAT_FILE()

    apqr_df = apqr.apqr_flat_file(APQR_FILE_PATH,apqr_output_file)

    df = apqr.merge(hmi_df,apqr_df)
    
    return df

In [65]:
file_name='D:\\OEE-GOA _USECASE\\GOA_UNIT4\\T+A 40_5\\data\\01_raw\\parameter_list.parquet'
product_name='AMLO BESY 5 MG AND TELMI 40 MG BILAYERED TABLETS'
parameter_no=[5,6,13,18,19]
APQR_FILE_PATH = 'D:\\OEE-GOA _USECASE\\GOA_UNIT4\\T+A 40_5\\APQR Jul-21 to Mar- 22 T+A 40 5.xlsx'
Merged_File = 'D:\\OEE-GOA _USECASE\\GOA_UNIT4\\T+A 40_5'
hmi_output_file = 'D:\\OEE-GOA _USECASE\\GOA_UNIT4\\T+A 40_5\\hmi_output_file.xlsx'
apqr_output_file = 'D:\\OEE-GOA _USECASE\\GOA_UNIT4\\T+A 40_5\\apqr_output_file.xlsx'
final_output_file='D:\\OEE-GOA _USECASE\\GOA_UNIT4\\T+A 40_5\\final_output_file.xlsx'

In [66]:
DataPreprocessingOE(file_name ,product_name,parameter_no,APQR_FILE_PATH,Merged_File,hmi_output_file,apqr_output_file,final_output_file)

(16435, 11)
{'station_1_reading_set': 'last', 'station_2_reading_set': 'last'}
[('Result', '% FINES') ('Result', 'BULK DENSITY')
 ('Result', 'LOSS ON DRYING') ('Result', 'TAPPED DENSITY')]


batch_id  station_1_reading_set Main compr.force MV kN  \
0    GD10733                                          2.96   
1    GD10734                                          2.97   
2    GD10757                                          3.00   
3    GD10773                                          2.90   
4    GD10774                                          2.86   
..       ...                                           ...   
125  GD20425                                          2.88   
126  GD20426                                          3.11   
127  GD20443                                          3.05   
128  GD20444                                          2.90   
129  GD20469                                          3.03   

     station_1_reading_set Pre compr. force MV kN  \
0                                            0.50   
1                                            0.52   
2                                            0.50   
3                                            0.50   
4                                            0.51   
..                                            ...   
125                                          0.50   
126                                          0.52   
127                                          0.50   
128                                          0.52   
129                                          0.50   

     station_1_reading_set Tabl. cyl. ht. main co. m  \
0                                               1.16   
1                                               1.16   
2                                               1.16   
3                                               1.16   
4                                               1.16   
..                                               ...   
125                                             1.16   
126                                             1.15   
127                                             1.18   
128                                             1.18   
129                                             1.18   

     station_1_reading_set Tabl. cyl. ht. pre co. mm  \
0                                               1.48   
1                                               1.47   
2                                               1.50   
3                                               1.47   
4                                               1.49   
..                                               ...   
125                                             1.52   
126                                             1.53   
127                                             1.56   
128                                             1.52   
129                                             1.57   

     station_1_reading_set Tabl. filling depth mm  \
0                                            2.99   
1                                            3.04   
2                                            3.00   
3                                            2.97   
4                                            2.90   
..                                            ...   
125                                          3.20   
126                                          3.20   
127                                          3.10   
128                                          3.03   
129                                          3.05   

     station_2_reading_set Main compr.force MV kN  \
0                                           16.10   
1                                           16.07   
2                                           15.99   
3                                           16.20   
4                                           16.15   
..                                            ...   
125                                         12.90   
126                                         13.30   
127                                         13.75   
128                                         13.90   
129                                         13.85   

  